In [16]:
import requests
from nagasaki.event_manager import EventManager
from nagasaki.logger import logger
from nagasaki.utils.common import round_decimals_down
from typing import Dict
from enum import Enum
from decimal import Decimal
from pydantic import BaseModel, validator
import json
from nagasaki.exceptions import CannotParseResponse, BitcludeClientException

class ActionEnum(str, Enum):
    SELL = "SELL"
    BUY = "BUY"


class MarketEnum(str, Enum):
    BTC = "BTC"
    PLN = "PLN"



class TransactionCreateRequestDTO(BaseModel):
    action: ActionEnum
    market1: MarketEnum
    market2: MarketEnum
    amount: Decimal
    rate: Decimal
    post_only: bool = True
    hidden: bool = True

    def get_request_params(self) -> Dict[str, str]:
        return {
            "method": "transactions",
            "action": self.action.value.lower(),
            "market1": self.market1.value.lower(),
            "market2": self.market2.value.lower(),
            "amount": str(self.amount),
            "rate": str(self.rate),
            "post_only": str(self.post_only),
            "hidden": str(self.hidden),
        }
    
    @validator("amount")
    def validate_amount(cls, v):
        rounded_v = round_decimals_down(v, 4)
        if rounded_v <=0:
            raise ValueError("Amount rounded down to 4 decimals must be greater than 0. got %.10f" % v)
        return rounded_v
    
    def __str__(self):
        return "%s %s/%s, amount: %.10f, rate: %.0f, post_only: %s, hidden: %s" % (
            self.action.value,
            self.market1.value,
            self.market2.value,
            self.amount,
            self.rate,
            self.post_only,
            self.hidden,
        )


class ActionDTO (BaseModel):
    action: ActionEnum
    order_id: str

    def __str__(self):
        return f"{self.action.value} order_id: {self.order_id}"

class TransactionCreateResponseDTO(BaseModel):
    success: bool
    code: str
    message: str
    actions: ActionDTO

    @validator('actions', pre=True)
    def fix_actions_structure(cls, v):
        if isinstance(v, dict):
            if 'sell' in v:
                return {'action': 'SELL', 'order_id': v['order']}
            elif 'buy' in v:
                return {'action': 'BUY', 'order_id': v['order']}
            else:
                raise ValueError("Unknown actions structure")
        else:
            raise ValueError("actions must be dict")
        
    def __str__(self):
        return f"success: {self.success}, code: {self.code}, message: '{self.message}', actions: ({self.actions})"


class BitcludeClient:
    def __init__(
        self,
        bitclude_url_base: str,
        bitclude_client_id: str,
        bitclude_client_key: str,
        event_manager: EventManager = None,
    ):
        self.bitclude_url_base = bitclude_url_base
        self.bitclude_client_id = bitclude_client_id
        self.bitclude_client_key = bitclude_client_key
        self.event_manager = event_manager or EventManager()

    def _get_auth_params(self):
        return {
            "id": self.bitclude_client_id,
            "key": self.bitclude_client_key,
        }

    def create_order(self, transaction: TransactionCreateRequestDTO) -> TransactionCreateResponseDTO:
        logger.info(transaction)
        transaction_params = transaction.get_request_params()
        auth_params = self._get_auth_params()
        params = {**transaction_params, **auth_params}
        response = requests.get(self.bitclude_url_base, params=params)
        try:
            response_json = response.json()
        except json.decoder.JSONDecodeError as json_decode_error:
            raise CannotParseResponse(response.text) from json_decode_error
        if response_json['success']:
            return TransactionCreateResponseDTO(**response_json)
        raise BitcludeClientException(response_json['message'])
        

bc = BitcludeClient(
    bitclude_url_base="https://api.bitclude.com",
    bitclude_client_id="xxx",
    bitclude_client_key="xxx",
)

t = TransactionCreateRequestDTO(
    action=ActionEnum.SELL,
    market1=MarketEnum.BTC,
    market2=MarketEnum.PLN,
    amount=Decimal("0.00011"),
    rate=Decimal(200_000),
)

response = bc.create_order(t)
print(response)


17:14:34.434 3325084434.py:112 SELL BTC/PLN, amount: 0.0001000000, rate: 200000, post_only: True, hidden: True


success: True, code: 5053, message: 'Order has been submited', actions: (SELL order_id: 11308169)
